# Real-World Data Demonstration

### Team Epsilon-Greedy Quants
#### Michael Lee, Nikat Patel, Jose Antonio Alatorre Sanchez

This notebook demonstrates what a user sees when the user runs the User Guide for Real-World Data Notebook

In [1]:
from environments.e_greedy import DeepTradingEnvironment, LinearAgent

import datetime
import numpy as np
import pandas as pd
import os
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.cla import CLA
import matplotlib.pyplot as plt
from matplotlib import cm
import copy
import quantstats as qs
qs.extend_pandas()

In [2]:
root = os.getcwd()
data_env = root+"/data_env/"

# Utility Functions

In [3]:
def _retrieve_asset_dict():
    # obtain close prices from parquet files of ETF price history
    root = os.getcwd()
    data_env = root+"/data_env/"
    files = [_ for _ in os.listdir(data_env) if "parquet" in _]
    assets_dict = {file: pd.read_parquet(data_env + "/" + file) for file in files}
    counter=0
    for key, value in assets_dict.items():
        if counter==0:
            main_index=value.index
    else:
        main_index=main_index.join(value.index,how="inner")
        
    for key, value in assets_dict.items():
        tmp_df=value.reindex(main_index)
        tmp_df=tmp_df.fillna(method='ffill')
        assets_dict[key]=tmp_df['close']
    return assets_dict

def build_portfolio_df(asset_dict):
    portfolio_df = pd.DataFrame()
    
    for key, value in assets_dict.items():
        key = key.split(".")[0]
        tmp_df = pd.DataFrame(data=value)
        tmp_df.columns=[key]
        portfolio_df = pd.concat([portfolio_df, tmp_df], axis=1)
        
    portfolio_df.index = pd.to_datetime(portfolio_df.index, errors='coerce')
    return portfolio_df

In [4]:
def plot_backtest(linear_agent_train, env_test, test_input, model_run, model):
    ## Create plot of backtest returns
    if not "backtest" in locals():
        backtest=None
    backtest, tmp_weights =linear_agent_train.backtest_policy(epoch=1,backtest=backtest, env_test=env_test, test_input=test_input)
    plt.figure(figsize=(8,4))
    plt.plot(backtest,color="blue")
    plt.gcf().autofmt_xdate()
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel("Date", fontsize = 10)
    plt.ylabel("Backtest", fontsize = 10)
    plt.title("Backtest on Test Data: "+ model,fontsize = 16)
    plt.savefig(root+'/temp_persisted_data/'+model_run+"_test_backtest_plot_"+model+'.png')
    tmp_weights.to_csv(root+'/temp_persisted_data/'+model_run+"_test_backtest_weights_"+model+'.csv')
    plt.show()
    return backtest

# Reviewing Real-World Data

In [5]:
# read a sample ETF

data_env_two_asset = root+ "/data_env/"

pd.read_parquet(data_env_two_asset+'SIZE.parquet').head()



,open,high,low,close,volume
index,,,,,
2017-01-03 05:00:00+00:00,67.24590,67.35794,66.97512,67.17120,26989
2017-01-04 05:00:00+00:00,67.77811,68.01154,67.77811,67.97419,8246
2017-01-05 05:00:00+00:00,68.17027,68.17027,67.61938,67.81546,3105
2017-01-06 05:00:00+00:00,67.74077,68.04888,67.64739,68.04888,3962
2017-01-09 05:00:00+00:00,67.80612,67.80612,67.61005,67.72208,1392


In [6]:
# create a portfolio
assets_dict = _retrieve_asset_dict()
portfolio_df = build_portfolio_df(assets_dict)

In [7]:
# create a train dataset and de-mean the time series

portfolio_df_train = portfolio_df[portfolio_df.index >= '2019-02-01']
portfolio_df_train = portfolio_df_train[portfolio_df_train.index <= '2020-02-01']
portfolio_df_train.sub(portfolio_df_train.mean())

portfolio_df_train.head()

,EEMV,EFAV,MTUM,QUAL,SIZE,USMV,VLUE
index,,,,,,,
2019-02-01 05:00:00+00:00,56.97454,65.81750,104.3037,80.92102,82.16029,53.53375,75.94203
2019-02-04 05:00:00+00:00,57.07571,66.04416,104.4988,81.45429,82.55833,53.91000,76.15092
2019-02-05 05:00:00+00:00,57.62489,66.40305,104.6647,81.98756,82.92724,54.06436,76.22688
2019-02-06 05:00:00+00:00,57.06607,65.94972,104.7428,81.84212,82.83987,53.98718,76.46426
2019-02-07 05:00:00+00:00,56.75777,65.49639,104.2744,81.02767,82.37387,53.87141,75.39131


In [8]:
# create a test dataset consisting of 6 months of data and de-mean the time series

portfolio_df_test = portfolio_df[portfolio_df.index >= '2020-04-16']
portfolio_df_test = portfolio_df_test[portfolio_df_test.index <= '2020-11-16']
portfolio_df_test.sub(portfolio_df_test.mean())

portfolio_df_test.head()

,EEMV,EFAV,MTUM,QUAL,SIZE,USMV,VLUE
index,,,,,,,
2020-04-16 04:00:00+00:00,48.20911,62.50506,116.0638,86.18110,75.67554,57.85078,65.36022
2020-04-17 04:00:00+00:00,48.88692,63.45106,117.9653,88.67091,78.40196,59.53394,67.65253
2020-04-20 04:00:00+00:00,48.48617,62.79575,116.7009,87.05402,77.06354,58.23692,66.08150
2020-04-21 04:00:00+00:00,47.54613,61.94337,112.3304,84.47494,74.79317,56.78149,63.82873
2020-04-22 04:00:00+00:00,48.56533,62.75141,115.3270,86.41917,76.31997,57.81118,65.00452


In [9]:
test_input_returns = portfolio_df_test.to_returns().dropna()
test_input_returns = test_input_returns.loc[(test_input_returns != 0).any(1)]


# Set Up Environment

In [10]:
# parameters related to the transformation of data, this parameters govern an step before the algorithm
out_reward_window=datetime.timedelta(days=7)

meta_parameters = {"in_bars_count": 1,
                   "out_reward_window":out_reward_window ,
                   "state_type":"in_window_out_window",
                   "risk_aversion":0,
                   "include_previous_weights":False}

# parameters that are related to the objective/reward function construction
objective_parameters = {"percent_commission": .001}

print("===Meta Parameters===")
print(meta_parameters)
print("===Objective Parameters===")
print(objective_parameters)

detrend=True

# create an environment and build features based on Real-World Dataset located in the "data_env" folder 
env = DeepTradingEnvironment.build_environment_from_dirs_and_transform(meta_parameters, objective_parameters,data_hash="real_data", data_dir="data_env", detrend=detrend)

number_of_assets = env.number_of_assets

===Meta Parameters===
{'in_bars_count': 1, 'out_reward_window': datetime.timedelta(days=7), 'state_type': 'in_window_out_window', 'risk_aversion': 0, 'include_previous_weights': False}
===Objective Parameters===
{'percent_commission': 0.001}


C:\Users\micha\anaconda3\envs\e599a_py37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\micha\anaconda3\envs\e599a_py37\lib\site-packages\statsmodels\tsa\holtwinters\model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
C:\Users\micha\anaconda3\envs\e599a_py37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\micha\anaconda3\envs\e599a_py37\lib\site-packages\statsmodels\tsa\holtwinters\model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
C:\Users\micha\anaconda3\envs\e599a_py

#### Split Features and Forward Returns into Training and Test sets

In [11]:
features = pd.read_parquet("temp_persisted_data/only_features_real_data")
features

,EEMV.parquet_log_returns,EFAV.parquet_log_returns,MTUM.parquet_log_returns,QUAL.parquet_log_returns,SIZE.parquet_log_returns,USMV.parquet_log_returns,VLUE.parquet_log_returns,EEMV.parquet_hw_trend,EEMV.parquet_hw_level,EEMV.parquet_hw_resid,...,USMV.parquet_hw_level_lag_1,USMV.parquet_hw_resid_lag_1,USMV.parquet_hw_demeaned_return_lag_1,USMV.parquet_hw_volatility_lag_1,VLUE.parquet_hw_trend_lag_1,VLUE.parquet_hw_level_lag_1,VLUE.parquet_hw_resid_lag_1,VLUE.parquet_hw_demeaned_return_lag_1,VLUE.parquet_hw_volatility_lag_1,bias
index,,,,,,,,,,,,,,,,,,,,,
2018-01-04 05:00:00+00:00,0.003545,0.005427,0.008348,0.004521,0.001779,0.003964,0.005525,0.853997,0.757334,0.694066,...,0.313287,0.475215,0.469748,0.000334,0.825701,0.720265,0.514148,0.446256,0.000295,1
2018-01-05 05:00:00+00:00,0.006573,0.002297,0.007997,0.006507,0.008025,0.005074,0.004562,0.874268,0.773000,0.702338,...,0.314318,0.485017,0.478085,0.001175,0.826959,0.725329,0.524536,0.469298,0.001216,1
2018-01-08 05:00:00+00:00,0.001277,-0.000270,0.002807,0.003061,0.001644,0.001686,0.001982,0.891746,0.788321,0.682771,...,0.316180,0.496796,0.481769,0.001731,0.829155,0.730991,0.531127,0.462344,0.000544,1
2018-01-09 05:00:00+00:00,-0.001437,0.000135,0.003731,0.003052,0.002344,0.000561,-0.000116,0.904653,0.802370,0.650754,...,0.318230,0.498528,0.463050,0.000080,0.831128,0.736651,0.529563,0.447409,0.000031,1
2018-01-10 05:00:00+00:00,-0.002721,-0.002027,-0.001025,-0.001877,-0.004811,-0.004874,0.000116,0.912432,0.814689,0.614831,...,0.320250,0.497054,0.457078,0.000025,0.832026,0.741818,0.522010,0.436442,0.000648,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-10 05:00:00+00:00,0.000856,0.006381,-0.012501,-0.001714,0.002722,0.007082,0.013513,0.810099,0.758448,0.689642,...,0.833321,0.603653,0.552428,0.029646,0.780858,0.657150,0.660463,0.624872,0.101811,1
2020-11-11 05:00:00+00:00,0.006310,0.008585,0.020153,0.003064,0.003005,0.004793,-0.000986,0.830056,0.771994,0.700141,...,0.842334,0.616774,0.484269,0.002966,0.804644,0.672354,0.682919,0.491396,0.008738,1
2020-11-12 05:00:00+00:00,-0.004430,-0.009575,-0.002721,-0.011950,-0.013448,-0.010364,-0.017783,0.843748,0.783617,0.656252,...,0.852480,0.619553,0.465000,0.000170,0.825077,0.687010,0.659346,0.407107,0.006197,1


In [12]:
features = pd.read_parquet("temp_persisted_data/only_features_real_data")

features_train = features[features.index >= '2019-02-01']
features_train = features_train[features_train.index <= '2020-02-01']

features_test = features[features.index >= '2020-04-16']
features_test = features_test[features_test.index <= '2020-11-16']


features_train.head()

,EEMV.parquet_log_returns,EFAV.parquet_log_returns,MTUM.parquet_log_returns,QUAL.parquet_log_returns,SIZE.parquet_log_returns,USMV.parquet_log_returns,VLUE.parquet_log_returns,EEMV.parquet_hw_trend,EEMV.parquet_hw_level,EEMV.parquet_hw_resid,...,USMV.parquet_hw_level_lag_1,USMV.parquet_hw_resid_lag_1,USMV.parquet_hw_demeaned_return_lag_1,USMV.parquet_hw_volatility_lag_1,VLUE.parquet_hw_trend_lag_1,VLUE.parquet_hw_level_lag_1,VLUE.parquet_hw_resid_lag_1,VLUE.parquet_hw_demeaned_return_lag_1,VLUE.parquet_hw_volatility_lag_1,bias
index,,,,,,,,,,,,,,,,,,,,,
2019-02-01 05:00:00+00:00,-0.004135,-0.000143,0.001872,0.004443,0.002958,0.002345,0.003382,0.862614,0.726692,0.609326,...,0.377093,0.574380,0.473081,0.000757,0.893021,0.583108,0.586058,0.454912,0.000089,1
2019-02-04 05:00:00+00:00,0.001774,0.003438,0.001869,0.006568,0.004833,0.007004,0.002747,0.868564,0.736292,0.602014,...,0.386603,0.566971,0.446022,0.000642,0.901893,0.596233,0.578064,0.435637,0.000712,1
2019-02-05 05:00:00+00:00,0.009576,0.005419,0.001586,0.006526,0.004459,0.002859,0.000997,0.878787,0.748087,0.631952,...,0.397083,0.572464,0.470057,0.000353,0.909409,0.609096,0.568538,0.432829,0.001072,1
2019-02-06 05:00:00+00:00,-0.009745,-0.006850,0.000746,-0.001776,-0.001054,-0.001429,0.003109,0.879714,0.756152,0.566833,...,0.407641,0.565109,0.446120,0.000632,0.914951,0.621270,0.554651,0.424844,0.002260,1
2019-02-07 05:00:00+00:00,-0.005417,-0.006898,-0.004482,-0.010001,-0.005641,-0.002147,-0.014131,0.875214,0.761813,0.528814,...,0.417356,0.545765,0.423786,0.004535,0.919485,0.633191,0.547573,0.437313,0.000655,1


In [13]:
forward_return_dates = pd.read_parquet("temp_persisted_data/forward_return_dates_real_data")

forward_return_dates_train = forward_return_dates[forward_return_dates.index >= '2019-02-01']
forward_return_dates_train = forward_return_dates_train[forward_return_dates_train.index <= '2020-02-01']

forward_return_dates_test = forward_return_dates[forward_return_dates.index > '2020-04-16']
forward_return_dates_test = forward_return_dates_test[forward_return_dates_test.index <= '2020-11-16']

forward_return_dates_train.head()

,"forward_return_7_days,_0:00:00"
index,
2019-02-01 05:00:00+00:00,2019-02-08 05:00:00+00:00
2019-02-04 05:00:00+00:00,2019-02-11 05:00:00+00:00
2019-02-05 05:00:00+00:00,2019-02-12 05:00:00+00:00
2019-02-06 05:00:00+00:00,2019-02-13 05:00:00+00:00
2019-02-07 05:00:00+00:00,2019-02-14 05:00:00+00:00


In [14]:
forward_returns = pd.read_parquet("temp_persisted_data/only_forward_returns_real_data")

forward_returns_train = forward_returns[forward_returns.index >= '2019-02-01']
forward_returns_train = forward_returns_train[forward_returns_train.index <= '2020-02-01']
forward_returns_train.sub(forward_returns_train.mean())

forward_returns_test = forward_returns[forward_returns.index >= '2020-04-16']
forward_returns_test = forward_returns_test[forward_returns_test.index <= '2020-11-16']
forward_returns_test.sub(forward_returns_test.mean())

forward_returns_train.head()

,"EEMV.parquet_forward_return_7_days,_0:00:00","EFAV.parquet_forward_return_7_days,_0:00:00","MTUM.parquet_forward_return_7_days,_0:00:00","QUAL.parquet_forward_return_7_days,_0:00:00","SIZE.parquet_forward_return_7_days,_0:00:00","USMV.parquet_forward_return_7_days,_0:00:00","VLUE.parquet_forward_return_7_days,_0:00:00"
index,,,,,,,
2019-02-01 05:00:00+00:00,-0.007018,-0.003731,0.003555,0.003954,0.002245,0.010452,-0.011003
2019-02-04 05:00:00+00:00,-0.013673,-0.009438,0.001681,-0.001547,0.001540,0.003579,-0.012219
2019-02-05 05:00:00+00:00,-0.020565,-0.008534,0.010535,0.003311,0.011239,0.007852,0.000000
2019-02-06 05:00:00+00:00,-0.014857,-0.003151,0.010247,0.010721,0.015821,0.011616,0.002980
2019-02-07 05:00:00+00:00,-0.006451,0.006056,0.013568,0.019026,0.019446,0.010924,0.014861


# Run Policy-Gradient Method Algorithms on Real-World Data

In [15]:
max_iter = 4001
model_run = "Full_Portfolio_lambda_0"
sample_observations = 2
plot_interval = 500

### REINFORCE

In [16]:
# create environment and run REINFORCE

env_reinforce_train=DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_reinforce_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_reinforce = LinearAgent(environment=env_reinforce_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_reinforce._load_benchmark(portfolio_df)
linear_agent_reinforce.REINFORCE_fit(max_iterations=max_iter, add_baseline=False, plot_every = plot_interval, train_input = portfolio_df_train, model_run = model_run, detrend=detrend, verbose=True)

  0%|                                                                                           | 0/36 [00:00<?, ?it/s]C:\Users\micha\anaconda3\envs\e599a_py37\lib\site-packages\numpy\lib\function_base.py:2474: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Users\micha\anaconda3\envs\e599a_py37\lib\site-packages\pypfopt\risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
  3%|██▎                                                                                | 1/36 [00:00<00:08,  4.35it/s]


LinAlgError: Eigenvalues did not converge

In [ ]:
# perform backtest 
backtest_reinforce = plot_backtest(linear_agent_reinforce, env_reinforce_test, portfolio_df_test, model_run, model="REINFORCE")
backtest_reinforce.to_csv('temp_persisted_data/'+model_run+'_backtest_reinforce.csv')

In [ ]:
(test_input_returns+1).cumprod().plot()

In [ ]:
(test_input_returns.sum(axis=1)+1).cumprod().plot()

### REINFORCE with Baseline

In [ ]:
# create environment and run REINFORCE with baseline
env_reinforce_baseline_train = DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_reinforce_baseline_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_reinforce_baseline = LinearAgent(environment=env_reinforce_baseline_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_reinforce_baseline._load_benchmark(portfolio_df)
linear_agent_reinforce_baseline.REINFORCE_fit(max_iterations=max_iter, add_baseline=True, plot_every = plot_interval, train_input = portfolio_df_train, model_run = model_run, detrend=detrend, verbose=True)

In [ ]:
# perform backtest 
backtest_reinforce_baseline = plot_backtest(linear_agent_reinforce_baseline, env_reinforce_baseline_test, portfolio_df_test, model_run, model="REINFORCE with Baseline")
backtest_reinforce_baseline.to_csv('temp_persisted_data/'+model_run+'_backtest_reinforce_baseline.csv')

### Actor-Critic

In [ ]:
# create environment and run Actor-Critic 

env_actor_critic_no_trace_train = DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_actor_critic_no_trace_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_actor_critic_no_trace = LinearAgent(environment=env_actor_critic_no_trace_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_actor_critic_no_trace._load_benchmark(portfolio_df)
linear_agent_actor_critic_no_trace.ACTOR_CRITIC_FIT(use_traces=False,max_iterations=max_iter, plot_every = plot_interval,train_input = portfolio_df_train, model_run = model_run, detrend=detrend, verbose=True)

In [ ]:
# perform backtest
backtest_actor_critic_no_trace = plot_backtest(linear_agent_actor_critic_no_trace, env_actor_critic_no_trace_test,  portfolio_df_test, model_run, model="Actor-Critic without Eligibility Traces")
backtest_actor_critic_no_trace.to_csv('temp_persisted_data/'+model_run+'_backtest_actor_critic_no_trace.csv')

### Actor-Critic with Eligibility Traces

In [ ]:
# create environment and run Actor-Critic with Eligibility Traces 
env_actor_critic_trace_train = DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_actor_critic_trace_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_actor_critic_trace = LinearAgent(environment=env_actor_critic_trace_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_actor_critic_trace._load_benchmark(portfolio_df)
linear_agent_actor_critic_trace.ACTOR_CRITIC_FIT(use_traces=True,max_iterations=max_iter, plot_every = plot_interval,train_input = portfolio_df_train, model_run = model_run, detrend=detrend, verbose=True)

In [ ]:
# perform backtest 
backtest_actor_critic_trace = plot_backtest(linear_agent_actor_critic_trace, env_actor_critic_trace_test,  portfolio_df_test, model_run, model="Actor-Critic with Eligibility Traces")
backtest_actor_critic_trace.to_csv('temp_persisted_data/'+model_run+'_backtest_actor_critic_trace.csv')